## Reachy's joints

In this notebook you will learn how to use Reachy's SDK to control Reachy's motors!

First, you need to connect to your Reachy. You will use *ReachySDK* for that.
The *ReachySDK* object takes three arguments at init, one of them is *host* (you don't need to worry about the others).

If you're working directly on Reachy, use 
```python
host='localhost'
```
If not, just get Reachy's IP address with
```bash
$ ifconfig
```
and put this addreess in the *host* argument.

In [1]:
from reachy_sdk import ReachySDK

In [2]:
reachy = ReachySDK(host='localhost')

You can use ReachySDK from any computer, as long as it and Reachy are connected to the same network.

There are 21 motors in Reachy's advanced version: 8 per arm, 3 for the orbita actuator used as the neck and 1 per antenna.

With the *reachy* you just instanciated, you can easily access each joints.

In [3]:
joints_names = [joint.name for joint in reachy.joints]
joints_names

['l_shoulder_pitch',
 'l_shoulder_roll',
 'l_arm_yaw',
 'l_elbow_pitch',
 'l_forearm_yaw',
 'l_wrist_pitch',
 'l_wrist_roll',
 'l_gripper',
 'r_shoulder_pitch',
 'r_shoulder_roll',
 'r_arm_yaw',
 'r_elbow_pitch',
 'r_forearm_yaw',
 'r_wrist_pitch',
 'r_wrist_roll',
 'r_gripper',
 'l_antenna',
 'r_antenna',
 'neck_disk_top',
 'neck_disk_middle',
 'neck_disk_bottom']

You can identify each arm joint with the documentation: 

<img src="https://raw.githubusercontent.com/pollen-robotics/reachy-sdk/notebooks_documentation/notebooks/images/arm_presentation/arm_schematic.png?token=AHZJ5IGUDBD5XMN4VMB2ZF3AMGTUW" alt="drawing" width="400"/>

Same with Obita:

<img src="images/dynamixel_speed/arm_shematic.png" alt="drawing" width="400"/>

For each joint, you can have acces to multiple information such as the position or the temperature, using the *registers()* method. 

In [8]:
reachy.r_shoulder_pitch.registers()

{'name': 'r_shoulder_pitch',
 'uid': 8,
 'present_position': 5.474197690505643,
 'temperature': 37.0,
 'compliant': True,
 'goal_position': 1.6060657291973135,
 'speed_limit': 0.0,
 'torque_limit': 100.0,
 'pid': (32.0, 0.0, 0.0)}

### Name/uid

Each joint has a name and a unique id associated so that when you want to get joints information, use the kinematics or use the *goto* function, you can either use the name or the id.

In [9]:
for joint in reachy.joints:
    print(f'{joint.name} got id {joint.uid}')

l_shoulder_pitch got id 0
l_shoulder_roll got id 1
l_arm_yaw got id 2
l_elbow_pitch got id 3
l_forearm_yaw got id 4
l_wrist_pitch got id 5
l_wrist_roll got id 6
l_gripper got id 7
r_shoulder_pitch got id 8
r_shoulder_roll got id 9
r_arm_yaw got id 10
r_elbow_pitch got id 11
r_forearm_yaw got id 12
r_wrist_pitch got id 13
r_wrist_roll got id 14
r_gripper got id 15
l_antenna got id 16
r_antenna got id 17
neck_disk_top got id 18
neck_disk_middle got id 19
neck_disk_bottom got id 20


### Present position

You can get the present position of each Reachy's joint.

This can be useful when you want to record a movement. For example you want to lift an object, you move the arm once yourself, store the *present_position* of each joint involved and replay it. This what we used to get the movements in our [Tictactoe application](https://www.youtube.com/watch?v=aGheS4HXEvI)!

Also if you want to teach movements to the robot with learning technics, storing the *present_position* of the joints will help you build a dataset.  

In [10]:
# for joint in reachy.joints:
print(f'Present position of r_shoulder_pitch joint is {"%.1f" %(reachy.r_shoulder_pitch.present_position)} degrees.')

Present position of r_shoulder_pitch joint is 5.5 degrees.


### Temperature

You also have access to each motors temperatures with the *temperature* attribute of each joint. 
This temperature should be checked to make sure that the robot can operate correctly. The temperature is also monitored internally in Reachy and fans are turned on when the joints are heating.  


See [Safety.ipynb notebook](https://github.com/pollen-robotics/reachy-sdk/blob/notebooks_documentation/notebooks/Safety.ipynb) for more information.

In [11]:
# for joint in reachy.joints:
print(f'Temperature of r_shoulder_pitch joint is {reachy.r_shoulder_pitch.temperature} Celsius degrees.')

Temperature of r_shoulder_pitch joint is 37.0 Celsius degrees.


### Compliant

Each Reachy's joint can be in two modes: **compliant** and **stiff**.

In the compliant mode, you can move the joint freely with your hands. 

In stiff mode, the joint will use its torque to stay in its present position. You can not move a motor in stiff mode with your hands. The only way to move the joint is by setting its *goal_position*. 

**Important:** Keep that in mind, if a joint does not move when you're changing its *goal_position*, it's very likely that it is in compliant mode.

Right now, Reachy's joints are in compliant mode, you can move each joint with your hands, try it!

Now let's turn Reachy's joints in stiff mode so that you can feel the difference. You should hear a small noise when the joints are in stiff mode, the noise should increase if you try to move with your hands a joint in stiff mode.

In [13]:
for joint in reachy.joints:
    joint.compliant = False

Try to move with the arms or the head, you should feel the joints resisting. 

Now, let's put the joints back to compliant mode.

In [14]:
for joint in reachy.joints:
    joint.compliant = True

### Goal position

As mentionned before, the *goal_position* attribute of a joint is what you need to set to move a joint.

For example, let's put the right arm in the right-angled position. 

**Important:** Make sure that the arm is placed on a table before and that there are no obstacles preventing Reachy from executing this movement. 

In [15]:
# First, put the arm's joints in stiff mode
for joint in reachy.joints[8:16]:
    joint.compliant = False

# Define what the goal_position of each joint will be
right_angle_position = {
    'r_shoulder_roll': 0,
    'r_arm_yaw': 0,
    'r_elbow_pitch': -90,
    'r_forearm_yaw': 0,
    'r_wrist_pitch': 0,
    'r_wrist_roll': 0,
    'r_gripper': 0,
}

#Set each goal_position
from operator import attrgetter

for name, goal in zip(right_angle_position.keys(), right_angle_position.values()):
    joint = attrgetter(name)(reachy)
    joint.goal_position = goal

Normally Reachy's right arm should be in the right-angled position!

Now we will put the arm back on the table by putting the joints in compliant mode. Place your hand below the arm to prevent it from falling hard on the table. 

In [16]:
for joint in reachy.joints[8:16]:
    joint.compliant = True

### Torque/Speed limit

The torque limit represents the percentage of maximum torque that the motor will use to stay in its present position when it is in stiff mode.
We set it to 100% on each joint.

In [17]:
reachy.r_shoulder_pitch.torque_limit

100.0

According to the motors documentation, the speed is in rpm (raw per minute).

<img src="https://raw.githubusercontent.com/pollen-robotics/reachy-sdk/notebooks_documentation/notebooks/images/arm_presentation/dynamixel_speed.png?token=AHZJ5IHAQCGSOAMGIYFRJQDAMGTNU" alt="drawing" width="600"/>



source: https://emanual.robotis.com/docs/en/dxl/ax/ax-18a/#ccw-angle-limit

We set the speed limit to 0 on each joint, meaning that the motors work at maximum speed. 

In [18]:
reachy.r_shoulder_pitch.speed_limit

0.0

**Important:** We recommand you NOT to change these values. When you want to make movements with Reachy, it's better to replay recorded trajectories or to work with the *goto* function from the API. *goto* computes a trajectory betweeen two points, see [Kinematics.ipynb](https://github.com/pollen-robotics/reachy-sdk/blob/notebooks_documentation/notebooks/Kinematics.ipynb) for more details.  

### PID

You can check and set the pid of each joint with the *pid* attribute.

In [19]:
reachy.r_shoulder_pitch.pid

(32.0, 0.0, 0.0)

If you are familiar with control theory, you'll know what it represents, if not don't worry just ignore it! 